<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Умножение матриц</a></span></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Алгоритм преобразования</a></span></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка алгоритма</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Защита персональных данных клиентов

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

## Загрузка данных

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

In [2]:
data = pd.read_csv('/datasets/insurance.csv')

In [3]:
data

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0
...,...,...,...,...,...
4995,0,28.0,35700.0,2,0
4996,0,34.0,52400.0,1,0
4997,0,20.0,33900.0,2,0
4998,1,22.0,32700.0,3,0


In [4]:
data.describe()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,0.499000,30.952800,39916.360000,1.194200,0.148000
std,0.500049,8.440807,9900.083569,1.091387,0.463183
min,0.000000,18.000000,5300.000000,0.000000,0.000000
25%,0.000000,24.000000,33300.000000,0.000000,0.000000
50%,0.000000,30.000000,40200.000000,1.000000,0.000000
75%,1.000000,37.000000,46600.000000,2.000000,0.000000
max,1.000000,65.000000,79000.000000,6.000000,5.000000


In [5]:
features = data.drop('Страховые выплаты', axis = 1)
target = data['Страховые выплаты']

## Умножение матриц

Для начала создадим модель.

In [6]:
class LinearRegression:
    def fit(self, train_features, train_target):
        X = np.concatenate((np.ones((train_features.shape[0], 1)), train_features), axis=1)
        y = train_target
        w = np.linalg.inv(X.T.dot(X)).dot(X.T).dot(y)
        self.w = w[1:]
        self.w0 = w[0]

    def predict(self, test_features):
        return test_features.dot(self.w) + self.w0

И обучим её на исходных данных.

In [7]:
model_original = LinearRegression()
model_original.fit(features, target)
predictions_original = model_original.predict(features)
mse_original = mean_squared_error(target, predictions_original)
print("MSE исходной модели:", mse_original)

MSE исходной модели: 0.12334688941710859


Создадим случайную обратимую матрицу.

In [8]:
new = np.random.normal(size=(features.shape[1],features.shape[1]))

Проверим её на обратимость.

In [9]:
try:
    np.linalg.inv(new)
    print('Матрица обратима')
except:
    print('Матрица необратима')

Матрица обратима


Замечательно. Перемножим исходную матрицу признаков и обратимую. 

In [10]:
test_features = features @ new

Обучим модель на полученном результате.

In [11]:
model_test = LinearRegression()
model_test.fit(test_features, target)
predictions_test = model_test.predict(test_features)
mse_test = mean_squared_error(target, predictions_test)
print("MSE тестовой модели:", mse_test)

MSE тестовой модели: 0.1233468894171088


**Ответ:** Как мы видим, результат практически не изменился.<br>
Попробуем это доказать. <br>

**Обоснование:**: Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

Для выражения $w'$ сначала в формулу обучения вместо $X$ подставим новую $X'$ = $XP$.

$$
w' = ((XP)^T XP)^{-1} (XP)^T y
$$

Вынесем $P$ отдельно.

$$
w' = P^{-1}(P^TX^TX)^{-1} P^TX^Ty 
$$

$$
w' = P^{-1}(X^TX)^{-1}(P^T)^{-1} P^TX^Ty
$$

Вспомним про то, что матрица умноженная на обратную равна единичной. 

$$
w' = P^{-1}(X^TX)^{-1} X^Ty
$$

Теперь заменим $(X^TX)^{-1} X^Ty$ на $w$ и получим, что

$$
w' = P^{-1}w
$$

Вернёмся к формуле предсказаний

$$
a'=X'w'
$$

$$
a'=XPP^{-1}w
$$

$$
a'= Xw
$$

$$
a'= a
$$

## Алгоритм преобразования

**Алгоритм**

1) С помощью np.random.normal генерируем случайную матрицу; <br>
2) Проверяем её на обратимость; <br>
3) Умножаем исходный набор признаков на случайную матрицу; <br>

Если потребуется из преобразованного набора данных получить исходный, умножаем его на обратную случайную матрицу.

**Обоснование**

Обоснование вытекает из пункта 2. <br>
Сначала мы его получили эмпирически, умножив исходные признаки на обратимую матрицу, затем - теоретически, проведя теоретические расчёты. <br>

## Проверка алгоритма

Вычислим R2-score исходной модели.

In [13]:
r2_original = r2_score(target, predictions_original)
print("R2 исходной модели:", r2_original)

R2 исходной модели: 0.4249455028666801


Теперь напишем функцию для шифрования данных.

In [19]:
def code_data(features):
    random_matrix = np.random.normal(size=(features.shape[1],features.shape[1])) #создаём случайную матрицу
    if np.linalg.det(random_matrix) != 0: #проверяем случайную матрицу на обратимость, сравнивая детерминант с нулем
        return features @ random_matrix 
    else:
        print('Случайная матрица обратима!')    

Проверим функцию на исходной матрице и вычислим r2_score получившейся зашифрованной:

In [24]:
coded_matrix = code_data(features)

model_valid = LinearRegression()
model_valid.fit(coded_matrix, target)
predictions_valid = model_valid.predict(coded_matrix)
r2_valid = r2_score(target, predictions_valid)
print("R2 модели с шифрованными данными:", r2_valid)

R2 модели с шифрованными данными: 0.42494550286661736


Успех!

## Чек-лист проверки

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Выполнен шаг 1: данные загружены
- [x]  Выполнен шаг 2: получен ответ на вопрос об умножении матриц
    - [x]  Указан правильный вариант ответа
    - [x]  Вариант обоснован
- [x]  Выполнен шаг 3: предложен алгоритм преобразования
    - [x]  Алгоритм описан
    - [x]  Алгоритм обоснован
- [x]  Выполнен шаг 4: алгоритм проверен
    - [x]  Алгоритм реализован
    - [x]  Проведено сравнение качества моделей до и после преобразования

# Выводы

В рамках выполнения проекта разработан метод шифрования данных страховой компании "Хоть потоп". <br>
Суть метода - в умножении матрицы исходных признаков на случайную обратимую матрицу. <br>

Произведена оценка качества модели до шифрования данных и после. MSE метрика практически не изменилась: <br>
Исходная модель: 0.12334688941710859 <br>
Тестовая модель: 0.1233468894171088 <br>

Математически обосновано, почему качество модели не изменилось. <br>

Написана функция для шифрования данных. Проверена R2-метрика до и после её применения. <br>
Исходная модель: 0.4249455028666801 <br>
Модель с шифрованными данными: 0.42494550286661736